In [1]:
from GAE_1 import Encoder, Decoder, GAE
from torchinfo import summary

import torch
from torch_geometric.utils import remove_self_loops, coalesce
from torch_geometric.data import Data
import numpy as np
import torch.optim as optim
import scipy
import os
import Get_data as Gd
from sklearn.model_selection import train_test_split
import torch.nn as nn

from tqdm import tqdm  
import matplotlib.pyplot as plt

/home/scheng1/miniconda3/envs/wl4023/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
# 检查是否有 GPU 可用
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

# 检查 GPU 的数量
num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")

# 获取每个 GPU 的名称
for i in range(num_gpus):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# 设置默认使用的 GPU 设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

GPU is not available
Number of GPUs available: 0
Using device: cpu


In [3]:
directory = '/data/Sibo_22Mar2024'
folders = [os.path.join(directory, f, 'hessian_') for f in os.listdir(directory) if f.startswith('case_')]
xyfile = "/data/Sibo_22Mar2024/case_0/hessian_/xy_coords.npy"
pos = torch.tensor(np.load(xyfile), dtype=torch.float32)
sparse_graph = scipy.sparse.load_npz('/data/Sibo_22Mar2024/sparse_matrix_fixed.npz')
indices = np.array(sparse_graph.nonzero())
values = sparse_graph.data
shape = sparse_graph.shape

# sparse edge tensor
edge_tensor = torch.sparse_coo_tensor(indices, values, shape, dtype=torch.float).coalesce()
edge_index = edge_tensor.indices()
edge_weight = edge_tensor.values()
print(edge_tensor.shape)
print(edge_index)
print(edge_weight)

torch.Size([97149, 97149])
tensor([[    0,     0,     1,  ..., 97148, 97148, 97148],
        [    1,     3,     0,  ..., 97145, 97146, 97147]])
tensor([1., 1., 1.,  ..., 1., 1., 1.])


In [4]:
dataset, length = Gd.get_all_nodes(folders[:-1])

train_np, val_np = train_test_split(dataset, test_size=0.2, shuffle=True)
train_np = np.expand_dims(train_np[:, :, 0], axis=2)
val_np = np.expand_dims(val_np[:, :, 0], axis=2)
train_tensor = torch.tensor(train_np, dtype=torch.float)
train_tensor_set = Gd.MyDataset(train_tensor)
val_tensor = torch.tensor(val_np, dtype=torch.float)
val_tensor_set = Gd.MyDataset(val_tensor)

In [5]:
edge_weight = edge_weight.unsqueeze(1)

In [6]:
num_mp_layers = [2, 2, 2]
num_clusters = [100, 5]
enc = Encoder(1, num_mp_layers, num_clusters, 16, 3)
summary(enc, input_data=(train_tensor[0], edge_index, edge_weight, pos))

Layer (type:depth-idx)                        Output Shape              Param #
Encoder                                       [5, 16]                   --
├─ModuleList: 1-5                             --                        (recursive)
│    └─Linear: 2-1                            [97149, 16]               32
├─ELU: 1-2                                    [97149, 16]               --
├─ModuleList: 1-5                             --                        (recursive)
│    └─Linear: 2-2                            [97149, 16]               272
├─ELU: 1-4                                    [97149, 16]               --
├─ModuleList: 1-5                             --                        (recursive)
│    └─Linear: 2-3                            [97149, 16]               272
├─ELU: 1-6                                    [97149, 16]               --
├─ModuleList: 1-45                            --                        (recursive)
│    └─ModuleList: 2-6                        --         

In [7]:
x, edge_index_1, edge_attr, edge_indices, edge_attrs, edge_indices_f2c, position, node_attrs, clusters = enc(train_tensor[0],
                                                                                                           edge_index, 
                                                                                                           edge_weight, pos)

In [8]:
dec = Decoder(1, num_mp_layers, 16, 3)
summary(dec, input_data=(x, edge_index_1, edge_attr, edge_indices, edge_attrs, edge_indices_f2c, position, node_attrs, clusters))

Layer (type:depth-idx)                        Output Shape              Param #
Decoder                                       [97149, 1]                --
├─ModuleList: 1-53                            --                        (recursive)
│    └─ModuleList: 2-3                        --                        (recursive)
│    │    └─Linear: 3-1                       [14, 16]                  32
├─ELU: 1-2                                    [14, 16]                  --
├─ModuleList: 1-53                            --                        (recursive)
│    └─ModuleList: 2-3                        --                        (recursive)
│    │    └─Linear: 3-2                       [14, 16]                  272
├─ELU: 1-4                                    [14, 16]                  --
├─ModuleList: 1-53                            --                        (recursive)
│    └─ModuleList: 2-3                        --                        (recursive)
│    │    └─Linear: 3-3                 

In [9]:
gae = GAE(1, 1, num_mp_layers, num_clusters, 16, 3)
summary(gae, input_data=(train_tensor[0], edge_index, edge_weight, pos))

Layer (type:depth-idx)                             Output Shape              Param #
GAE                                                [97149, 1]                --
├─Encoder: 1-1                                     [5, 16]                   --
│    └─ModuleList: 2-5                             --                        (recursive)
│    │    └─Linear: 3-1                            [97149, 16]               32
│    └─ELU: 2-2                                    [97149, 16]               --
│    └─ModuleList: 2-5                             --                        (recursive)
│    │    └─Linear: 3-2                            [97149, 16]               272
│    └─ELU: 2-4                                    [97149, 16]               --
│    └─ModuleList: 2-5                             --                        (recursive)
│    │    └─Linear: 3-3                            [97149, 16]               272
│    └─ELU: 2-6                                    [97149, 16]               --
│    └